In [1]:
from multitask_dataset import SingerMultiTaskDataset
from torch.utils.data import DataLoader
from torch.optim import Adam
from models import HuBERTMultiHead
import pandas as pd

# import torch
# torch.autograd.detect_anomaly(True)

In [2]:
train_audio_folder = '/media/maximos/9C33-6BBD/data/melos_singers/Rebetika_vowels/train/'
csv_path = '/media/maximos/9C33-6BBD/data/melos_singers/features/multitask_targets.csv'

In [3]:
feats = pd.read_csv(csv_path, delimiter=',')
features_list = list(feats.columns)
print(features_list)
del(features_list[:2])
print(features_list)

['Unnamed: 0', 'names', 'singer_id', 'Pitch', 'SpectralCentroid', 'SpectralSpread', 'SpectralSkewness', 'SpectralKurtosis', 'SpectralFlatness', 'SpectralCrest', 'SpectralSlope', 'SpectralDecrease', 'SpectralRollOff', 'SpectralVariation', 'SpectralFlux', 'HarmonicSpectralDeviation', 'Tristimulus_1', 'Tristimulus_2', 'Tristimulus_3', 'HarmonicOddToEvenRatio', 'Inharmonicity', 'HarmonicEnergy', 'NoiseEnergy', 'Noisiness', 'HarmonicToNoiseEnergy', 'PartialsToNoiseEnergy', 'F1_Hz', 'F2_Hz', 'F3_HZ', 'F4_Hz', 'Rate', 'Depth', 'Regularity']
['singer_id', 'Pitch', 'SpectralCentroid', 'SpectralSpread', 'SpectralSkewness', 'SpectralKurtosis', 'SpectralFlatness', 'SpectralCrest', 'SpectralSlope', 'SpectralDecrease', 'SpectralRollOff', 'SpectralVariation', 'SpectralFlux', 'HarmonicSpectralDeviation', 'Tristimulus_1', 'Tristimulus_2', 'Tristimulus_3', 'HarmonicOddToEvenRatio', 'Inharmonicity', 'HarmonicEnergy', 'NoiseEnergy', 'Noisiness', 'HarmonicToNoiseEnergy', 'PartialsToNoiseEnergy', 'F1_Hz', '

In [4]:
task_labels_num_out = {}
for i in range(1, len(features_list)-3, 1):
    task_labels_num_out[features_list[i]] = 1
# add singer identification
task_labels_num_out['singer_id'] = feats['singer_id'].max()+1 # accounting for zero
print(task_labels_num_out)

{'Pitch': 1, 'SpectralCentroid': 1, 'SpectralSpread': 1, 'SpectralSkewness': 1, 'SpectralKurtosis': 1, 'SpectralFlatness': 1, 'SpectralCrest': 1, 'SpectralSlope': 1, 'SpectralDecrease': 1, 'SpectralRollOff': 1, 'SpectralVariation': 1, 'SpectralFlux': 1, 'HarmonicSpectralDeviation': 1, 'Tristimulus_1': 1, 'Tristimulus_2': 1, 'Tristimulus_3': 1, 'HarmonicOddToEvenRatio': 1, 'Inharmonicity': 1, 'HarmonicEnergy': 1, 'NoiseEnergy': 1, 'Noisiness': 1, 'HarmonicToNoiseEnergy': 1, 'PartialsToNoiseEnergy': 1, 'F1_Hz': 1, 'F2_Hz': 1, 'F3_HZ': 1, 'F4_Hz': 1, 'singer_id': 6}


In [5]:
model = HuBERTMultiHead(task_labels_num_out=task_labels_num_out)

You are using a model of type hubert to instantiate a model of type wav2vec2. This is not supported for all configurations of models and can yield errors.
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
training_data = SingerMultiTaskDataset(train_audio_folder, csv_path)

In [7]:
print(len(training_data))

336


In [8]:
print(training_data[235])

{'input_values': array([ 9.6231786e-05, -4.8065151e-04, -1.0562916e-03, ...,
        2.1723036e-02,  3.0537494e-02,  3.0633982e-02], dtype=float32), 'labels': {'Unnamed: 0': 235, 'names': 'keti_grei_e_7', 'singer_id': 3, 'Pitch': 0.7209138518538807, 'SpectralCentroid': 0.5698297871797114, 'SpectralSpread': 0.5059747324580315, 'SpectralSkewness': 0.40996711878204867, 'SpectralKurtosis': 0.17471883980858025, 'SpectralFlatness': 2.1642168877359462e-07, 'SpectralCrest': 0.19977750826511775, 'SpectralSlope': 0.5214147424361909, 'SpectralDecrease': 0.02280188370441561, 'SpectralRollOff': 0.21781613652017218, 'SpectralVariation': 0.9633068641347643, 'SpectralFlux': 0.10157637350694174, 'HarmonicSpectralDeviation': 0.35987722172305736, 'Tristimulus_1': 0.29529396714096523, 'Tristimulus_2': 0.6616446652813145, 'Tristimulus_3': 0.16561083301404245, 'HarmonicOddToEvenRatio': 0.003796548057622091, 'Inharmonicity': 0.10887323108745597, 'HarmonicEnergy': 0.24651267309414998, 'NoiseEnergy': 0.4692333

In [9]:
dataloader = DataLoader(training_data, batch_size=8, shuffle=True, collate_fn=model.collate_fn)

In [10]:
print(dataloader.__dict__)

{'dataset': <multitask_dataset.SingerMultiTaskDataset object at 0x742144559580>, 'num_workers': 0, 'prefetch_factor': None, 'pin_memory': False, 'pin_memory_device': '', 'timeout': 0, 'worker_init_fn': None, '_DataLoader__multiprocessing_context': None, '_dataset_kind': 0, 'batch_size': 8, 'drop_last': False, 'sampler': <torch.utils.data.sampler.RandomSampler object at 0x74213c208bf0>, 'batch_sampler': <torch.utils.data.sampler.BatchSampler object at 0x7420c90e84a0>, 'generator': None, 'collate_fn': <bound method HuBERTMultiHead.collate_fn of HuBERTMultiHead(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
      

In [11]:
b = next(iter(dataloader))

In [12]:
print(len(b[0]['input_values']))
print(len(b[0]['attention_mask']))
print(len(b[1]))
print(b[1])

8
8
33
{'Unnamed: 0': [114, 117, 2, 133, 286, 213, 254, 106], 'names': ['kazantzidis_i_5', 'kazantzidis_i_8', 'bellou_a_3', 'kazantzidis_old_a_9', 'ninou_u_3', 'keti_grei_a_12', 'keti_grei_u_7', 'kazantzidis_i_12'], 'singer_id': [2, 2, 0, 2, 4, 3, 3, 2], 'Pitch': [0.39692045289014205, 0.0626757632559464, 0.21904812775121937, 0.2441134872026566, 0.22345971504630882, 0.35831408163231426, 0.4961932795789526, 0.3198237253521693], 'SpectralCentroid': [0.5956305636719355, 0.5797184008174251, 0.560793287984073, 0.43236903248499314, 0.0748293342222268, 0.3789232577511787, 0.23530275669125422, 0.9028203593469135], 'SpectralSpread': [0.5557710901311825, 0.6525194978577975, 0.5687656186835492, 0.4657550754144305, 0.12657206931787549, 0.41745796307360317, 0.4450094724239343, 0.7948371011643081], 'SpectralSkewness': [0.19785289228703504, 0.26317222060875595, 0.3782914699514791, 0.44745988216438537, 0.5707278917530184, 0.4410572889383226, 0.6882963479499091, 0.2680919880332829], 'SpectralKurtosis': 

In [13]:
model.train()
optimizer = Adam(model.parameters())

In [14]:
print(model.hubert.encoder.layers[0].attention.k_proj.weight)

Parameter containing:
tensor([[ 0.0764, -0.0356,  0.0319,  ...,  0.0229,  0.0379, -0.0324],
        [-0.0147,  0.0039, -0.0659,  ...,  0.0499,  0.0458, -0.0392],
        [ 0.1157,  0.0316,  0.0097,  ..., -0.1181, -0.0043, -0.0881],
        ...,
        [-0.0829, -0.0190, -0.0334,  ..., -0.1305, -0.0136,  0.0721],
        [ 0.0076, -0.1277, -0.0363,  ..., -0.0170,  0.0036, -0.0242],
        [-0.0481, -0.0188, -0.0242,  ...,  0.0573, -0.0350,  0.0071]],
       device='cuda:0', dtype=torch.float16, requires_grad=True)


In [15]:
b[0]['attention_mask']

[array([1, 1, 1, ..., 0, 0, 0], dtype=int32),
 array([1, 1, 1, ..., 0, 0, 0], dtype=int32),
 array([1, 1, 1, ..., 0, 0, 0], dtype=int32),
 array([1, 1, 1, ..., 0, 0, 0], dtype=int32),
 array([1, 1, 1, ..., 0, 0, 0], dtype=int32),
 array([1, 1, 1, ..., 0, 0, 0], dtype=int32),
 array([1, 1, 1, ..., 1, 1, 1], dtype=int32),
 array([1, 1, 1, ..., 0, 0, 0], dtype=int32)]

In [16]:
y = model(
    audio_normalized=b[0]['input_values'],
    attention_mask=b[0]['attention_mask'],
    labels=b[1],
    output_attentions=False,
    output_hidden_states=False,
    return_dict=True
)

task_name:  Pitch
tensor([0.3970, 0.0627, 0.2190, 0.2441, 0.2235, 0.3584, 0.4961, 0.3198],
       device='cuda:0', dtype=torch.float16)
tensor([0.9980, 0.3689, 0.0444, 0.9990, 0.9980, 0.5581, 0.9976, 0.6943],
       device='cuda:0', dtype=torch.float16, grad_fn=<SqueezeBackward0>)
loss 0:  tensor(0.2607, device='cuda:0', dtype=torch.float16,
       grad_fn=<MseLossBackward0>)


In [17]:
print(y)

SequenceClassifierOutput(loss=tensor(0.2607, device='cuda:0', dtype=torch.float16,
       grad_fn=<MseLossBackward0>), logits=tensor([[0.9980],
        [0.3689],
        [0.0444],
        [0.9990],
        [0.9980],
        [0.5581],
        [0.9976],
        [0.6943]], device='cuda:0', dtype=torch.float16,
       grad_fn=<SigmoidBackward0>), hidden_states=tensor([[[ 0.1004, -0.3003,  0.1274,  ..., -0.2177, -0.0836,  1.0762],
         [ 0.3643, -0.6777,  0.0834,  ..., -0.3359, -0.0428,  1.0273],
         [ 0.1174, -0.4790,  0.1021,  ..., -0.1317,  0.1111,  1.1299],
         ...,
         [ 0.0166,  0.2343, -0.3572,  ..., -0.2710, -0.1511, -0.1639],
         [ 0.0454, -0.0725, -0.3977,  ..., -0.4651, -0.2917, -0.4336],
         [ 0.1163,  0.2458, -0.4246,  ..., -0.3918, -0.0404, -0.6367]],

        [[ 0.2688, -0.0598,  0.0507,  ..., -0.0206,  0.0519,  1.5430],
         [ 0.1919, -0.0804, -0.1019,  ...,  0.0634,  0.1296,  0.9790],
         [ 0.3528, -0.0920, -0.1510,  ..., -0.0350,  0.02

In [18]:
optimizer.zero_grad()
y.loss.backward()# retain_graph=True
optimizer.step()

In [38]:
model.projectors['Pitch'].weight.grad

tensor([[-1.3603e-02, -1.1650e-02, -2.2030e-03,  ..., -1.9150e-03,
         -1.5335e-03, -8.0933e-02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 6.3837e-05,  2.2686e-04,  6.9666e-04,  ...,  0.0000e+00,
          1.2094e-04,  3.5439e-03],
        ...,
        [ 1.9028e-02,  1.6449e-02,  2.7561e-03,  ...,  2.7275e-03,
          2.1343e-03,  1.1176e-01],
        [ 5.8929e-02,  5.0446e-02,  9.5291e-03,  ...,  8.2855e-03,
          6.6452e-03,  3.5034e-01],
        [ 5.3131e-02,  4.5502e-02,  8.5983e-03,  ...,  7.4730e-03,
          5.9891e-03,  3.1592e-01]], device='cuda:0', dtype=torch.float16)

In [44]:
print(model.hubert.encoder.layers[11].final_layer_norm.weight)

Parameter containing:
tensor([0.2285,   -inf,    inf, 0.2190,    nan,   -inf, 0.4187, 0.7261,   -inf,
          -inf,    nan, 0.2480,    nan, 0.7188,   -inf,    inf, 0.4805, 0.2067,
           nan,    inf,    nan,    inf,    nan,    inf,   -inf,   -inf,   -inf,
           inf,    inf, 0.2324,   -inf,   -inf, 0.2808, 0.2074,    nan, 0.2744,
          -inf, 0.6509,   -inf, 0.2573,    inf, 0.2151, 0.1991, 0.3291,   -inf,
           nan,   -inf,   -inf,    nan, 0.2323, 0.1743, 0.2756,   -inf, 0.2712,
           nan, 0.2502,    inf, 0.2969,    inf,   -inf,    inf,    inf,   -inf,
           inf, 0.2585, 0.1826,    inf, 0.6992, 0.2211, 0.2751,    nan,   -inf,
        0.6831,    inf, 0.4600,   -inf, 0.2278,    inf,    nan, 0.2216,    nan,
           inf, 0.2339,   -inf,   -inf,   -inf,   -inf, 0.3628, 0.2344, 0.4543,
        0.2007, 0.1808,   -inf,   -inf, 0.7349,   -inf,    inf, 0.2468, 0.2367,
           nan, 0.2107,   -inf,   -inf,   -inf,    nan,    inf, 0.2832,    nan,
          -inf, 0.

In [19]:
print(len(model.hubert.encoder.layers))

12


In [24]:
print(model.hubert.encoder.layers[11].feed_forward.output_dense.weight)
# print(model.hubert.encoder.layers[0].attention.k_proj.weight.grad)

Parameter containing:
tensor([[inf, inf, inf,  ..., inf, inf, inf],
        [inf, inf, inf,  ..., inf, inf, inf],
        [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
        ...,
        [-inf, -inf, -inf,  ..., -inf, -inf, -inf],
        [inf, inf, inf,  ..., inf, inf, inf],
        [-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:0',
       dtype=torch.float16, requires_grad=True)
